In [ ]:
# This presentation was delivered using [RISE](https://github.com/damianavila/RISE).

# Clear all outputs and run this cell before starting the presentation.
# The Meter stuff defines things that will be used in demos but presented only
# as Markdown.
# The Plots and Images stuff is purely to get past the annoying JIT delay.

struct Meter{N}
    value::Float64
end
Base.:*(x::Meter{M}, y::Meter{N}) where {M,N} = Meter{M+N}(x.value * y.value)
Base.:/(x::Meter{M}, y::Meter{N}) where {M,N} = Meter{M-N}(x.value / y.value)
Base.:*(x::Real, y::Meter{N}) where N = Meter{N}(x * y.value)
Base.:+(x::Meter{N}, y::Meter{N}) where N = Meter{N}(x.value + y.value)
Base.show(io::IO, x::Meter{N}) where N = print(io, x.value, "m^", N)
Base.show(io::IO, x::Meter{1}) = print(io, x.value, "m")
Base.show(io::IO, x::Meter{0}) = print(io, x.value)

const m = Meter{1}(1)

using Plots; plotlyjs()

fntsm = Plots.font("sans-serif", 10.0)
fntlg = Plots.font("sans-serif", 14.0)
Plots.default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm)
Plots.default(size=(720,540))
display(plot(linspace(0, 5π, 200), sin))

using Images, TestImages
img = testimage("lighthouse")

# <center>The Julia programming language</center>
<br>
<center>![julia logo](figures/julia_logo.png)</center>

### <center>Timothy E. Holy</center>
<center>Department of Neuroscience</center>
<center>Washington University in St. Louis</center>

# What is Julia and where did it come from?

A free and open-source general purpose programming language with an emphasis on scientific computing


Created by Jeff Bezanson, Stefan Karpinski, Viral Shah, and Alan Edelman (all at MIT) in 2009

"Went public" in 2012 thanks to a now-famous blog post:
![blog post](figures/whywecreatedjulia.png)

# "I've never heard of it, why should I care?"

There are thought to be [thousands of programming languages](https://en.wikipedia.org/wiki/List_of_programming_languages#C), though not many are serious contenders for scientific programming (Matlab, Python, R, Mathematica, Octave, now Julia)

Julia is growing fast:

|  |  |
| --- | --- |
| ![stats](figures/growingfast.png) | ![stats](figures/top10languages.png) |

# Why is Julia succeeding?

* Interactive & dynamic
* Elegant language design
* A focus on _composable_ and _reusable_ code
* Outstanding performance

# Interactive and dynamic

In the box above, I demo (one after another):
```
2+3
a = [1 2; 3 4]
a*a
```

# Elegant language design (trivial example)

```julia
function cartesian(r, θ, ϕ = 0)
    x, y, z = r*sin(θ)*cos(ϕ), r*sin(θ)*sin(ϕ), r*cos(θ)
end
```

Above I mention simple stuff: the use of unicode, the nice syntax for default arguments

# Elegant language design (typical example)

```julia
if !isfile(graph_file)
    # namespaces!
    rq = Requests.get("http://api.brain-map.org/api/v2/structure_graph_download/1.json")
    data = JSON.parse(String(rq.data))
    # built-in dictionaries! chained indexing that actually works!
    hierarchy = data["msg"][1]

    # closes files when done, even if there's an error in the middle!
    open(graph_file, "w") do io
        write(io, JSON.json(hierarchy))
    end
end
```


The above was an almost randomly-chosen block of code that I then looked at and asked whether it improved on the expectations I had before I adopted Julia. Even in a short block of code, I found several.

# Elegant language design (complex example)

<span style="font-size: 0.5em;">
"A Linear Time Algorithm for Computing Exact Euclidean Distance Transforms of
Binary Images in Arbitrary Dimensions"
Maurer, Qi, Raghavan 2003
</span>

<center>![bwdist](figures/bwdist_pseudocode.png)</center>

The above algorithm would be complicated to implement in most languages. In particular, the last block of code is a set of `d` nested `for`-loops. In many languages you might have to write a separate implementation for each dimensionality, if you wanted to make it fast.

# Elegant language design (complex example, Julia implementation)

```julia
function computeft!(F, I, w, jpost::CartesianIndex{K}) where K
    null = nullindex(F)
    if K == ndims(I)-1
        # Fig. 2, lines 2-8
        for i1 in indices(I, 1)
            F[i1, jpost] = ifelse(I[i1, jpost], CartesianIndex(i1, jpost), null)
        end
    else
        # Fig. 2, lines 10-12
        for i1 in indices(I, ndims(I) - K)
            computeft!(F, I, w, CartesianIndex(i1, jpost))
        end
    end
    # Fig. 2, lines 14-20
    indspre = ftfront(indices(F), jpost)  # discards the trailing indices of F
    for jpre in CartesianRange(indspre)
        voronoift!(F, I, w, jpre, jpost)
    end
    F
end


nullindex(A::AbstractArray{T,N}) where {T,N} = typemin(Int)*one(CartesianIndex{N})

ftfront(inds, reference::NTuple{K}) where K = ftfront(front(inds), reference)
ftfront(inds::NTuple{K}, reference::NTuple{K}) where K = inds
```


The thing to observe here is that the Julia implementation is basically the same as the pseudocode, and not obviously any longer. The CartesianRange/CartesianIndex stuff allows you to create nested `for`-loops.

# Composable and reusable code

```julia
struct Meter{N}
    value::Float64
end
Base.:*(x::Meter{M}, y::Meter{N}) where {M,N} = Meter{M+N}(x.value * y.value)
Base.:/(x::Meter{M}, y::Meter{N}) where {M,N} = Meter{M-N}(x.value / y.value)
Base.:*(x::Real, y::Meter{N}) where N = Meter{N}(x * y.value)
Base.:+(x::Meter{N}, y::Meter{N}) where N = Meter{N}(x.value + y.value)
Base.show(io::IO, x::Meter{N}) where N = print(io, x.value, "m^", N)
Base.show(io::IO, x::Meter{1}) = print(io, x.value, "m")
Base.show(io::IO, x::Meter{0}) = print(io, x.value)

const m = Meter{1}(1)
```

Composable = put your design effort into "small" objects that work together, and assemble them into big and beautiful projects.

Here we're going to create a new type of number and use it in computations. To keep it familiar, we'll make it behave like a physical unit. (The experts here might notice that the multiplication rule is not inferrable, at least with current versions of Julia, but on balance I decided that simplicity was better than perfection. It could be fixed with a `@generated` function.)

I don't go into great detail talking about these: I just point out a few highlights, like "we're defining arithmetic rules here. There is one thing to note: the `N` is the exponent (power) attached to the meters, and so these numbers follow the rules that you've come to expect from physical units: meters * meters = meter^2, and so on. The `show` methods just affect the display of quantities. The `m` constant will be handy for creating objects of this type." That's about it.

In [ ]:
x, y = 3m, 2m

In the box above I try simple stuff first: I show that `x+y` works, then `x*y`. Then I point out that `x^5` works even though we didn't define a special method for it. Then I create `a = [1m 2m; 3m 4m]` and show that matrix multiplication works even though we didn't have to write an algorithm for multiplying matrices of this new kind of number. (Again the afficianodos will get bothered by that `Any`, but the `@generated` version would solve that.)

In [ ]:
using Colors
c = colormatch(509)   # emission peak of Green Fluorescent Protein (in nm)

Starting here I begin another illustration, one showing how nice it is to work with colors as a single entity. The `showtext` method below is just so people can appreciate what's happening under the hood, since Colors/IJulia/Jupyter display it as a color by default. There's an important consequence for image processing: it means that we have the equation "1 array element" == "1 pixel," and that's true for grayscale and color images alike. In many other languages you'd need an `m`-by-`n`-by-3 array for color, and in some cases you might not be able to tell whether that 3rd dimension is for color or whether it's a "thin slab" of a genuinely 3d image or short grayscale movie sequence.

In [ ]:
# Define a function for displaying objects as "raw text"
showtext(x) = show(STDOUT, MIME("text/plain"), x)

showtext(c)

In [ ]:
showtext(RGB(c))

In [ ]:
using Images, TestImages
img = testimage("lighthouse")

In [ ]:
size(img)

In [ ]:
img[250,60]

In [ ]:
showtext(img[250,60])       # N0f8 = 8-bit fixed point number

In [ ]:
sizeof(img)/length(img)

The observation above implies that you can pack these colors tightly: only 3 bytes/color. The abstraction does not add overhead.

This allows you to write "generic" code (you don't need separate versions for color vs grayscale images)

# Performance

### A note on benchmarks

These benchmarks are designed to test *programming paradigms* (loops, recursion, string parsing, etc.) and not *user experience*. Your actual experience can deviate from the impression you might get from these benchmarks.

For matrices `A` and `B`, `A*B` is fast in every language, because most use the same underlying Fortran libraries.

<center> ![benchmarks](figures/benchmarks.png) </center>

# What makes some languages slow?

### Computing all comes down to interpreting 0s and 1s

```julia
julia> bits(5)  # show the binary representation of "5" (64-bit integer)
"0000000000000000000000000000000000000000000000000000000000000101"
```

Now we get to the most technical part. I mention that from a certain perspective, Julia isn't fast: Julia's big trick is that it avoids being slow :-). This is a useful distinction, because it helps one realize that efficient computing comes from getting out of the way of the hardware, letting it do its job as intended.

I tried to come up with a way to motivate type systems for people who may not be familiar with the concepts, and for whom reading LLVM IR or assembly might not exactly be second-nature. Because integer and floating point numbers use different binary representation and are operated on by separate transistor paths in the CPU, I found the following route to be fairly easy to explain.

```julia
julia> bits(5.0)  # show the binary representation of "5.0" (64-bit floating-point number)
"0100000000010100000000000000000000000000000000000000000000000000"

julia> reinterpret(Int, 5.0)
4617315517961601024
```
(This is not specific to Julia, this is true for any programming language.)

$\rightarrow$ Every collection of bits also comes with a *type* that tells how those 0s and 1s should be interpreted.

# What makes some languages slow?

### You have to use different "methods" depending on the *type*

Doing math with integers and floating point numbers:

<center> ![cpu](figures/cpu.gif) </center>

`x + y` gets translated into `add_using_the_integer_unit(x, y)` or `add_using_the_fpu(x, y)` depending on the types of `x` and `y`.

# What makes some languages slow?

Implementing `x + y` in a slow language:

```julia
function mysum(x, y)
    if isa(x, Float64) && isa(y, Float64)
        return add64_using_the_fpu(x, y)
    elseif isa(x, Float32) && isa(y, Float32)
        return add32_using_the_fpu(x, y)
    elseif isa(x, Int64) && isa(y, Int64)
        return add64_using_the_integer_unit(x, y)
    elseif isa(x, Int32) && isa(y, Int32)
        return add64_using_the_integer_unit(x, y)
    else
        # convert everything to Float64
        x64, y64 = Float64(x), Float64(y)
        return add64_using_the_fpu(x64, y64)
    end
end
```

Doing the addition might take a nanosecond or less. Determining the type might take tens of nanoseconds.

# How do slow languages work around this?

### Vectorization: check once, and then do the same operation many times

```julia
# This function would be written in a fast language (C), so that the loops below are fast
function mysum(x::Array, y::Array)
    if eltype(x) == Float64 && eltype(y) == Float64
        z = similar(x)
        for i = 1:length(x)
            z[i] = add64_using_the_fpu(x[i], y[i])   # these are fast
        end
    else
        ...
    end
end
```

# Disadvantages of relying on vectorization

* Everything fast must be written in C. You're at the mercy of others to write something in a language you may not understand (the "two language problem").

* Performance problems: `z[i] = sin(x[i]) + cos(y[i])` can be computed without temporary storage, but the (naive) vectorized version `z = sin(x) + cos(y)` creates two temporaries.

* Sometimes the above can be worked around, but sometimes not (myth of the "sufficiently smart compiler").

* Sometimes vectorization is "pretty," but sometimes your code is simpler and clearer if you can just write loops.

But if vectorization works for you, keep using it!

# How does C solve this problem?

* The programmer specifies the type of *everything*.
* Since all types are known, the entire program has predictable types: *compile* once, *run* many times.
* Downsides: painful & inflexible.

# How does Julia solve this problem?

Julia figures out types *automatically* ("type inference"), and compiles an optimized version on-the-fly ("just in time").

The key point is that Julia was designed to make the types as *predictable* as possible:

```julia
function myfunction(x, y)  # if Julia knows the types of the inputs...
    z = f(g(h(x))) + k(y)  # ...it can peer inside dependent functions...
    ...
    result = a .* b + c    # ...and keep track of types all the way to the end
end
```

It will build (compile) a whole new version of `myfunction` for each combination of input types---and each one will be optimized to run quickly.

# Why don't other languages do the same thing?

They try (and tens or hundreds of millions have been spent trying). But the languages weren't designed from the outset to make this "easy," so it frequently breaks down.

That's why most scientific languages rely ultimately on code written in C.

Most impressive exception: JavaScript.

# Julia's key strengths: a summary

* Interactive & dynamic (just-in-time compiling)

* Outstanding performance (type inference)

* Elegant language design

* A focus on _composable_ and _reusable_ code (focus on small building blocks, which you assemble into bigger things---this is very hard if you rely on vectorization)

# A downside of the type system


Above, I start out with a blank cell and show that `sqrt(2)` works but that `sqrt(-2)` throws an error. That you can work around this with `sqrt(complex(-2))`.

# When type inference isn't enough

This code works, but it's not inferrable:
```julia
function mysqrt(x)
    if x < 0
        return sqrt(complex(x))   # returns a complex number
    end
    sqrt(x)                       # returns a real number
end
```
This hurts performance.

Julia developers chose to make the "core" function fast, but if you prefer the behavior above you can add it yourself---but it will cost you in terms of performance.

I say that this tradeoff is something that seems basically inevitable, and that while you often can get away without thinking (much) about types, ultimately you can't have Julia's great performance without them intruding occasionally. I point out that if this frustrates you, unlike some of the negatives that come next it's something that is unlikely to change---it's a permanent consequence of the design choices of the language.

# Julia's key weaknesses

* Julia is not yet at 1.0: the language is still changing

* Type inference and compiling is slow

* It's still young and so the ecosystem is still growing

# Do *not* switch to Julia (yet) if...

* you're perfectly happy with what you're using now
* everything you do involves simple stuff like matrix algebra and summing columns of numbers
* you can't tolerate the idea of rewriting some aspects of your code because language changes broke it
* you use huge packages that simply aren't available in Julia

# Strongly consider switching to Julia if...

* the performance of what you're using now is just killing you
* limitations of the language you use now are making your code too ugly to maintain
* you have free time on your hands and want to try something new

# Switch now or after 1.0? Pros and cons

* Pro of waiting: no need to rewrite code
* Pro of testing it now: if you notice something ugly about Julia, there may still time to fix it

# How to assess whether Julia is ready for you (yet)

Packages:
* Lists: http://genieframework.com/packages and https://juliaobserver.com/packages
* "Curated" list: http://svaksha.github.io/Julia.jl/

Play with it on a *small* project and see how you like it

# To learn more about the Julia language

* https://julialang.org/
* https://en.wikibooks.org/wiki/Introducing_Julia
* Performance tips: https://docs.julialang.org/en/stable/manual/performance-tips
* Discussions: https://discourse.julialang.org/

Acknowledgments and anything else "personal" to the author were deleted prior to distribution.